# Lectura: Transformaciones Comunes y Técnicas de Optimización en Spark

Cuando trabajas con DataFrames de PySpark para el procesamiento de datos, es importante conocer los dos tipos de transformaciones: estrechas y amplias. Las transformaciones estrechas en Spark funcionan dentro de las particiones sin reorganizar los datos entre ellas. Se aplican localmente a cada partición, evitando el intercambio de datos. Por otro lado, las transformaciones amplias en Spark implican redistribuir y reorganizar los datos entre las particiones, lo que a menudo conduce a operaciones más complejas y que consumen más recursos.

Para entender mejor este concepto, echemos un vistazo a la siguiente ilustración.

![https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-BD0225EN-SkillsNetwork/images/Picture1.png](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-BD0225EN-SkillsNetwork/images/Picture1.png)

Dentro de las transformaciones estrechas, los datos se transfieren sin ejecutar operaciones de reorganización de datos.

![https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-BD0225EN-SkillsNetwork/images/Picture2.png](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-BD0225EN-SkillsNetwork/images/Picture2.png)

Las transformaciones amplias implican la reorganización de datos entre particiones

**Ejemplos de transformaciones estrechas**

Las transformaciones estrechas se pueden comparar con realizar operaciones simples en conjuntos de datos distintos. Considera tener varios tipos de datos en contenedores separados. Puedes realizar acciones en cada contenedor de datos o mover datos entre contenedores de forma independiente sin requerir interacción o transferencia. Ejemplos de transformaciones estrechas incluyen modificar piezas individuales de datos, seleccionar elementos específicos o combinar dos contenedores de datos.

1. **Map:** Aplicar una función a cada elemento del conjunto de datos.



In [ ]:
from pyspark import SparkContext
sc = SparkContext("local", "MapExample")
data = [1, 2, 3, 4, 5]
rdd = sc.parallelize(data)
mapped_rdd = rdd.map(lambda x: x * 2)
mapped_rdd.collect() # Output: [2, 4, 6, 8, 10]




1. **Filter:** Seleccionar elementos basados en una condición específica.



In [ ]:
from pyspark import SparkContext
sc = SparkContext("local", "FilterExample")
data = [1, 2, 3, 4, 5]
rdd = sc.parallelize(data)
filtered_rdd = rdd.filter(lambda x: x % 2 == 0)
filtered_rdd.collect() # Output: [2, 4]




**3. Union:** Combinar dos conjuntos de datos con el mismo esquema.



In [ ]:
from pyspark import SparkContext
sc = SparkContext("local", "UnionExample")
rdd1 = sc.parallelize([1, 2, 3])
rdd2 = sc.parallelize([4, 5, 6])
union_rdd = rdd1.union(rdd2)
union_rdd.collect() # Output: [1, 2, 3, 4, 5, 6]




**Ejemplos de transformaciones amplias**

Las transformaciones amplias se pueden comparar con tareas realizadas en equipo donde se necesita información de diferentes grupos para concluir. Imagina que tienes un grupo de amigos, cada uno con una pieza de un rompecabezas. Para armar el rompecabezas, podrías necesitar intercambiar piezas entre tus amigos para que todo encaje. Estas tareas son un buen ejemplo de transformación amplia. Tales tareas pueden ser un poco más complicadas porque todos necesitan colaborar y mover piezas.

1. **GroupBy:** Agregar datos basados en una clave específica.



In [ ]:
from pyspark import SparkContext
sc = SparkContext("local", "GroupByExample")
data = [("apple", 2), ("banana", 3), ("apple", 5), ("banana", 1)]
rdd = sc.parallelize(data)
grouped_rdd = rdd.groupBy(lambda x: x[0])
sum_rdd = grouped_rdd.mapValues(lambda values: sum([v[1] for v in values]))
sum_rdd.collect() # Output: [('apple', 7), ('banana', 4)]




1. **Join:** Combinar dos conjuntos de datos basados en una clave común.



In [ ]:
from pyspark import SparkContext
sc = SparkContext("local", "JoinExample")
rdd1 = sc.parallelize([("apple", 2), ("banana", 3)])
rdd2 = sc.parallelize([("apple", 5), ("banana", 1)])
joined_rdd = rdd1.join(rdd2)
joined_rdd.collect() # Output: [('apple', (2, 5)), ('banana', (3, 1))]





**3. Sort:** Reorganizar los datos basados en un criterio específico.



In [ ]:
from pyspark import SparkContext
sc = SparkContext("local", "SortExample")
data = [4, 2, 1, 3, 5]
rdd = sc.parallelize(data)
sorted_rdd = rdd.sortBy(lambda x: x, ascending=True)
sorted_rdd.collect() # Output: [1, 2, 3, 4, 5]




Las transformaciones amplias son similares a reorganizar y redistribuir datos entre diferentes grupos. Imagina tener conjuntos de datos que quieres combinar u organizar de una nueva manera. Sin embargo, esta tarea no es tan sencilla como trabajar con un solo conjunto de datos. Necesitas coordinar y mover datos entre estos conjuntos, lo que implica más complejidad. Por ejemplo, fusionar dos conjuntos de datos basados en un atributo común requiere reorganizar los datos entre ellos, lo que la convierte en una transformación amplia en ingeniería de datos.

**PySpark DataFrame: Transformaciones comunes basadas en reglas**

La API de DataFrame en PySpark ofrece varias transformaciones basadas en reglas predefinidas. Estas transformaciones están diseñadas para mejorar cómo se ejecutan las consultas y aumentar el rendimiento general. Veamos algunas transformaciones comunes basadas en reglas.

1. **Predicate pushdown:** Empujar las condiciones de filtrado más cerca de la fuente de datos antes de procesar para minimizar el movimiento de datos.
2. **Constant folding:** Evaluar expresiones constantes durante la compilación de la consulta para reducir la computación durante el tiempo de ejecución.
3. **Column pruning:** Eliminar columnas innecesarias del plan de consulta para mejorar la eficiencia del procesamiento.
4. **Join reordering:** Reorganizar las operaciones de unión para minimizar el tamaño de los datos intermedios y mejorar el rendimiento de la unión.



In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
# Create a Spark session
spark = SparkSession.builder.appName("RuleBasedTransformations").getOrCreate()
# Sample input data for DataFrame 1
data1 = [
("Alice", 25, "F"),
("Bob", 30, "M"),
("Charlie", 22, "M"),
("Diana", 28, "F")
]
# Sample input data for DataFrame 2
data2 = [
("Alice", "New York"),
("Bob", "San Francisco"),
("Charlie", "Los Angeles"),
("Eve", "Chicago")
]
# Create DataFrames
columns1 = ["name", "age", "gender"]
df1 = spark.createDataFrame(data1, columns1)
columns2 = ["name", "city"]
df2 = spark.createDataFrame(data2, columns2)
# Applying Predicate Pushdown (Filtering)
filtered_df = df1.filter(col("age") > 25)
# Applying Constant Folding
folded_df = filtered_df.select(col("name"), col("age") + 2)
# Applying Column Pruning
pruned_df = folded_df.select(col("name"))
# Join Reordering
reordered_join = df1.join(df2, on="name")
# Show the final results
print("Filtered DataFrame:")
filtered_df.show()
print("Folded DataFrame:")
folded_df.show()
print("Pruned DataFrame:")
pruned_df.show()
print("Reordered Join DataFrame:")
reordered_join.show()
# Stop the Spark session
spark.stop()


Filtered DataFrame:
+-----+---+------+
| name|age|gender|
+-----+---+------+
|  Bob| 30|     M|
|Diana| 28|     F|
+-----+---+------+

Folded DataFrame:
+-----+---------+
| name|(age + 2)|
+-----+---------+
|  Bob|       32|
|Diana|       30|
+-----+---------+

Pruned DataFrame:
+-----+
| name|
+-----+
|  Bob|
|Diana|
+-----+

Reordered Join DataFrame:
+-------+---+------+-------------+
|   name|age|gender|         city|
+-------+---+------+-------------+
|  Alice| 25|     F|     New York|
|    Bob| 30|     M|San Francisco|
|Charlie| 22|     M|  Los Angeles|
+-------+---+------+-------------+





**Técnicas de optimización utilizadas en Spark SQL**

1. **Predicate pushdown:** Aplica un filtro a DataFrame "df1" para seleccionar solo las filas donde la columna "edad" sea mayor que 25.
2. **Constant folding:** Realiza una operación aritmética en la columna "edad" en el folded_df, añadiendo un valor constante de 2.
3. **Column pruning:** Selecciona solo la columna "nombre" en el pruned_df, eliminando columnas innecesarias del plan de consulta.
4. **Join reordering:** Realiza una unión entre df1 y df2 en la columna "nombre", permitiendo que Spark reordene la unión para mejorar el rendimiento.

**Técnicas de optimización basadas en costos en Spark**

Spark emplea técnicas de optimización basadas en costos para mejorar la eficiencia de la ejecución de consultas. Estos métodos implican estimar y analizar los costos asociados con las consultas, lo que lleva a decisiones más informadas que resultan en un rendimiento mejorado.

1. **Adaptive query execution:** Ajusta dinámicamente el plan de consulta durante la ejecución basándose en estadísticas en tiempo de ejecución para optimizar el rendimiento.
2. **Cost-based join reordering:** Optimiza el orden de unión basado en los costos estimados de diferentes rutas de unión.
3. **Broadcast hash join:** Optimiza las uniones de tablas pequeñas transmitiendo una tabla a todos los nodos, lo que reduce la redistribución de datos.
4. **Shuffle partitioning and memory management:** EGestiona eficientemente la redistribución de datos durante operaciones como groupBy y agregación y optimiza el uso de la memoria.

Al utilizar estos métodos, Spark se esfuerza por ofrecer capacidades de procesamiento de datos eficientes y escalables. Es esencial comprender la aplicación efectiva de estas transformaciones y optimizaciones para lograr el mejor rendimiento posible de las consultas y la utilización óptima de los recursos del sistema.



In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
# Create a Spark session
spark = SparkSession.builder.appName("CostBasedOptimization").getOrCreate()
# Sample input data for DataFrame 1
data1 = [
("Alice", 25),
("Bob", 30),
("Charlie", 22),
("Diana", 28)
]
# Sample input data for DataFrame 2
data2 = [
("Alice", "New York"),
("Bob", "San Francisco"),
("Charlie", "Los Angeles"),
("Eve", "Chicago")
]
# Create DataFrames
columns1 = ["name", "age"]
df1 = spark.createDataFrame(data1, columns1)
columns2 = ["name", "city"]
df2 = spark.createDataFrame(data2, columns2)
# Enable adaptive query execution
spark.conf.set("spark.sql.adaptive.enabled", "true")
# Applying Adaptive Query Execution (Runtime adaptive optimization)
optimized_join = df1.join(df2, on="name")
# Show the optimized join result
print("Optimized Join DataFrame:")
optimized_join.show()
# Stop the Spark session
spark.stop()


Optimized Join DataFrame:


+-------+---+-------------+
|   name|age|         city|
+-------+---+-------------+
|  Alice| 25|     New York|
|    Bob| 30|San Francisco|
|Charlie| 22|  Los Angeles|
+-------+---+-------------+





En este ejemplo, creamos dos DataFrames (**df1** y **df2**) con datos de entrada de muestra. Luego, activamos la función de ejecución de consultas adaptativas configurando el parámetro de configuración **"spark.sql.adaptive.enabled"** en **"true"**.La Ejecución de Consultas Adaptativas permite que Spark ajuste el plan de consulta durante la ejecución basándose en estadísticas en tiempo de ejecución.

El código realiza una unión entre **df1** y **df2** en la columna "name". La ejecución de consultas adaptativas de Spark ajusta dinámicamente el plan de consulta basándose en estadísticas en tiempo de ejecución, lo que puede resultar en un mejor rendimiento.

## **Author(s)**

Raghul Ramesh

![https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-BD0225EN-SkillsNetwork/images/footer%20logo.png](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-BD0225EN-SkillsNetwork/images/footer%20logo.png)